<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Recomendando-compras-para-e-commerce" data-toc-modified-id="Recomendando-compras-para-e-commerce-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Recomendando compras para e-commerce</a></span></li><li><span><a href="#Passos-que-daremos" data-toc-modified-id="Passos-que-daremos-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Passos que daremos</a></span><ul class="toc-item"><li><span><a href="#Subscription" data-toc-modified-id="Subscription-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Subscription</a></span></li><li><span><a href="#Criando-um-produto" data-toc-modified-id="Criando-um-produto-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Criando um produto</a></span></li><li><span><a href="#Código-para-enviarmos-imagens" data-toc-modified-id="Código-para-enviarmos-imagens-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Código para enviarmos imagens</a></span><ul class="toc-item"><li><span><a href="#As-constantes-do-código-acima" data-toc-modified-id="As-constantes-do-código-acima-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>As constantes do código acima</a></span></li></ul></li><li><span><a href="#Código-que-retorna-os-K-itens-mais-similares-a-um-item-qualquer" data-toc-modified-id="Código-que-retorna-os-K-itens-mais-similares-a-um-item-qualquer-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Código que retorna os K itens mais similares a um item qualquer</a></span></li></ul></li></ul></div>

# Recomendando compras para e-commerce

Suponha que você possua um e-commerce que venda roupas e acessórios de moda. Para todo cliente que entra no seu site, você gostaria de mostrar sugestões de compras, para isso, pegamos os 5 itens mais similares ao item que ele está procurando e mostramos na página do site como sugestão.<br>

O que chamamos de similaridade aqui é uma medida de quanto um item se parece com o outro. Assumimos que se o cliente estiver procurando por uma camiseta de time, ele iria gostar de ver outras camisetas de time e não um óculos de natação.<br>

De uma maneira bem simples, o que queremos fazer é um sistema de recomendação _"Content Base"_, ou seja, baseado apenas no item que o cliente está procurando no momento, não estamos levando em conta o histórico completo dele e nem as compras que ele efetuou.<br>

Para isso, iremos usar como base a plataforma [mycelia](https://mycelia.developer.azure-api.net/).

# Passos que daremos

Vamos mostrar todos os passos e códigos que precisamos usar para interagir com o [mycelia](https://mycelia.developer.azure-api.net/) e ter um retorno dos **k** itens mais similares a um item qualquer que quisermos. Os passos do tutorial serão:

1. Fazer o subscription na plataforma
2. Criar um produto para termos uma chave de autenticação.
3. Criar e rodar o código para enviarmos imagens ao mycelia
4. Retornar e visualizar os k itens mais similares a um item qualquer

##  Subscription

Para fazermos uso da similaridade por imagens, precisamos fazer o subscription na plataforma. Primeiro, entre no portal do [mycelia](https://mycelia.developer.azure-api.net/) e clique em **Sign Up** como indicado na imagem.

![title](tutorial_images/sign_up.PNG)

Você cairá na página da imagem abaixo. Preencha com os seus dados, escolha uma senha e clique em **Sign Up** no final da página.

![title](tutorial_images/sign_up_2.PNG)

Após esse processo, você receberá um email para confirmar sua inscrição na plataforma. Coonfirme e pronto, você está habilitado para usar tudo que o [mycelia](https://mycelia.developer.azure-api.net/) oferece e outras coisas mais.

## Criando um produto

Após o processo acima, você estará apto a usar a plataforma. Faça o login e depois clique em **Products**, como indicado na imagem abaixo.

![title](tutorial_images/products_sign.PNG)

Você cairá na página abaixo. Precisaremos da aprovação do *admin* para termos acesso ao produto *Unlimited*. Veja que no começo apenas o *Starter* aparece.

![title](tutorial_images/products_starter.PNG)

Após a aprovação do *admin*, você verá o produto *Unlimited* aparecendo. Clique nele.

![title](tutorial_images/product_unlimited.PNG)

Na próxima página crie um nome para seu produto e depois clique em **Subscribe**. Veja a imagem abaixo.

![title](tutorial_images/subscribe_unli.PNG)

Após clicar em Subscribe você será direcionado para a página abaixo. Seu produto já está criado, o *admin* irá autorizá-lo e ele deverá mudar para o **State Active**, só assim para conseguirmos usá-lo. Você usará sua **Primary Key**, ela será a chave de autenticação do seu produto.

![title](tutorial_images/subs_keys.PNG)

Pronto, vamos colocar a mão na massa e rodar alguns códigos.

## Código para enviarmos imagens 

Nós precisamos enviar as imagens do nosso portfólio de produtos para o [mycelia](https://mycelia.developer.azure-api.net/). A plataforma irá pegar essas imagens e criar representações numéricas delas. Afinal, um computador não entende o que é uma imagem, ele não tem olho, precisamos mostrá-las em formato de números para ele. Agora, o como a plataforma pega essas imagens e transforma em números ficará para um outro tutorial, onde explicaremos passo a passo os modelos que estão sendo usados e os papers que serviram de referência.<br>

O script abaixo contém todas as funções que precisamos para enviar imagens que estão num diretório qualquer. Por padrão, enviamos imagens em formato .jpg, mas você pode mudar. Todas as funções estão com docstrings e você pode usá-las como bem entender.<br> 

In [1]:
"""
--- send_requests.py ---
Script que envia imagens de um diretorio para a plataforma
mycelia.
"""
import base64
import glob
import json
import requests
import os
import pandas as pd
from pathlib import Path


def rand_path_images(path_folder: str='data/images/*.jpg', num_images: int=5_000, 
                     read_all: bool=False) -> np.array:
    """Seleciona aleatoriamente uma quantidade n de imagens de um diretorio
    
    Params
    ----------
    path_folder (str): path do diretorio onde estao as imagens. Defaults to 'data/images/*.jpg'
    
    num_images (int): quantidade de imagens para selecionar no diretorio. Defaults to 5_000
    
    read_all (bool): opcao para selecionar e enviar todas as imagens de um diretorio. Defaults to False
    
    to_csv (bool): opcao para criarmos um csv com as imagens que enviaremos. Isso serve para 
    batermos o index de cada imagem e saber o que estamos comparando. Defaults to True
    
    Return
    ----------
    array_path_images (np.array): array com os paths das imagens que selecionamos aleatoriamente
    
    Exemplos
    ----------
    Suponha que temos um diretorio com o esquema abaixo
    +-- data
        +-- image_1.jpg
        +-- image_2.jpg
        +-- ...........
        +-- image_99.jpg
        +-- image_100.jpg
        
 
    >>> array_path_images = rand_path_images(path_folder='data/*.jpg', num_images=2)
    >>> print(array_path_images)
    array(['data\\image_55.jpg',
           'data\\image_18.jpg'], dtype='<U37')
           
    """
    np.random.seed(42)
    
    if read_all:
        array_path_images = np.array(glob.glob(path_folder))
    else:
        array_path_images = np.array(glob.glob(path_folder))
        array_rand_idx = np.random.choice(len(array_path_images) - 1, num_images, replace=False)
        array_path_images = array_path_images[array_rand_idx]
    return array_path_images


def encode_images(img_file: str):
    """Faz o encode das imagens
    
    Params
    ---------- 
    img_file (str): string com o caminho completo de uma imagem
    
    Return
    ----------
    encoded_string (str): string que representa a imagem encodada
    
    Exemplos
    ----------
    Suponha uma imagem com o caminho 'data/image_1.jpg'
    
    >>> img = 'data/image_1.jpg'
    >>> encod_img = encode_images(img)
    >>> print(encod_img)
    /9j/4RVxRXhpZgAASUkqAAgAAAAMAAABAwABAAAAOAQAAAEBAwABAAAAoAUAAAIBA
    
    """
    with open(img_file, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8") 
    return encoded_string

    
def create_data_list(array_path_images: np.array) -> list:
    """Cria uma lista de dicionarios com o indice e a string da imagem encodada
    
    Params
    ---------- 
    array_path_images (np.array): array com o path completo das imagens que usaremos
    
    Return
    ----------
    img_arr (list[dict]): retorna uma list de dicionarios. Este fica na forma:
    {'index': index_da_imagem, "image_base_64": imagem_encodada}.
    Os index das imagens devem ser unicos. O valor da chave "image_base_64" eh a saida
    da funcao encode_images()
    
    Exemplos
    ----------
    >>> PATH_FOLDER = 'data/*.jpg'
    >>> NUM_IMAGES = 2
    
    >>> array_path_images = rand_path_images(path_folder=PATH_FOLDER, num_images=NUM_IMAGES)
    >>> img_arr = create_data_list(array_path_images=array_path_images)
    >>> print(img_arr)
    [{'index': 0, 'image_base64': '/9j/4RVxRXhpZgAASUkqAAgAAAA'}, 
     {'index': 1, 'image_base64': '/5h/1NXxPOqlOqFFCAkzWWeWWWW'}]
     
    """
    img_arr = []
    for image in array_path_images:
            encoded_img = encode_images(image)
            idx = Path(image).stem
            img_arr.append({"index": idx, "image_base64": encoded_img})
            
    return img_arr


def send_requests(imgs: list, db_name: str, headers: dict, batch_size: int=1_000) -> None:
    """Envia os requests para o mycelia em batchs
    
    Params
    ---------- 
    imgs (list): list de dicts. Nada mais do que a saida da funcao create_data_list()
    
    db_name (str): nome que daremos para a nossa base
    
    headers (dict): dicionario com a key sendo a string 'Auth' e o value sendo 
    a Primary Key em formato de string. Para acessar sua primary key, va na plataforma
    mycelia e acesse Profile, aperte em Show e cole a Primary Key.
    
    batch_size (int): tamanho do batch que enviaremos para a plataforma mycelia
    
    Return
    ----------
    None
    
    Exemplos
    ----------
    >>> PATH_FOLDER = 'data/*.jpg'
    >>> NUM_IMAGES = 5_000
    >>> HEADER = {'Auth': '1237ydajndndwajd1'}
    >>> DB_NAME = "fashion_data_base"
    >>> BATCH_SIZE = 1_000
    
    >>> array_path_images = rand_path_images(path_folder=PATH_FOLDER, num_images=NUM_IMAGES)
    >>> imgs = create_data_list(array_path_images=array_path_images)
    >>> send_requests(imgs=imgs, db_name=DB_NAME, headers=HEADER, batch_size=BATCH_SIZE)
    
    """
    for i in range(0, len(imgs), batch_size):
        begin = i
        end = i + batch_size
        r = requests.post(f"https://mycelia.azure-api.net/image/{db_name}", data=json.dumps(imgs[begin:end]), 
            headers=headers)
        print(f"Response: {r.text}")
        
        
# -------------------------------------------------------------------------------
# -------------------------------------------------------------------------------
# -------------------------------------------------------------------------------

if __name__ == '__main__':
    PATH_FOLDER = 'data/fashion-dataset/images/*.jpg'
    NUM_IMAGES = 1_000 # quantidade de imagens que pegaremos aleatoriamente do folder acima
    HEADER = {'Auth': 'sua_primary_key_mycelia'}
    DB_NAME = "nome_qualquer_que_vc_escolher"
    BATCH_SIZE = 1_000 # tamanho dos batchs que mandaremos

    array_path_images = rand_path_images(path_folder=PATH_FOLDER, num_images=NUM_IMAGES)

    imgs = create_data_list(array_path_images=array_path_images)
    send_requests(imgs=imgs, db_name=DB_NAME, headers=HEADER, batch_size=BATCH_SIZE)

### As constantes do código acima

1. PATH_FOLDER: é o caminho do diretório onde as imagens se encontram. O final *.jpg significa que iremos ler apenas arquivos com a extensão .jpg, caso queira ler .png, basta mudar.<br><br>

2. NUM_IMAGES: suponha que você tenha um diretório com 50.000 imagens, mas gostaria de mandar apenas um _sample_ de 10.000, essa constante irá controlar essa quantidade. Caso queira pegar o diretório inteiro, apenas passe True para o parâmetro **read_all** da função **rand_path_images()**.<br><br>

3. HEADER: é um dicionário da forma {'Auth': 'primary_key_string'}, ou seja, o valor é a sua **Primary Key.**<br><br>

4. DB_NAME: nome do database que criaremos na plataforma mycelia<br><br>

5. BATCH_SIZE: por questões de infraestrutura, fica mais fácil mandar as imagens em _batchs_, estes são apenas lotes com uma quantidade de dados. Aqui no exemplo, estamos mandando _batchs_ com 1.000 imagens cada

## Código que retorna os K itens mais similares a um item qualquer

Após o [mycelia](https://mycelia.developer.azure-api.net/) transformar as imagens do nosso portfólio em representações numéricas, podemos usar essas representações e compará-las, ver quais estão mais próximas umas das outras. Por sorte, a plataforma também já faz isso, só precisamos interagir com ela.

In [28]:
"""
--- retrieve_k_similar.py ---
Script que retorna k itens mais similares a um item x qualquer.
"""
import json
import pandas as pd
import requests


def list_k_similar_distance(db_name: str, idx_item: int, top_k: int) -> list:
    """Retorna uma lista de dicionarios com o index e a distancia dos k itens mais
    proximos do item procurado pelo cliente.
    
    Params
    ----------
    db_name (str): string com o nome do db que voce criou na plataforma mycelia
    
    idx_item (int): index do item que o cliente esta procurando no momento
    
    top_k (int): quantidade k de itens mais proximos do item procurado pelo cliente
    
    Return
    ----------
    list_index_distance (list): lista de dicionarios com index e distancia de cada item procurado
    
    Exemplos
    ----------
    Vamos retornar os 5 itens mais proximos do item de index 1439 da base beers
    
    >>> TOP_K = 5 
    >>> IDX_ITEM = 1439 
    >>> DB_NAME = 'beers'
    >>> list_index_distance = list_k_similar_distance(db_name=DB_NAME, idx_item=IDX_ITEM, top_k=TOP_K)
    >>> print(list_index_distance)
    [{'index': 1439, 'distance': 0.0},
     {'index': 2148, 'distance': 11.271553039550781},
     {'index': 1824, 'distance': 17.47911834716797},
     {'index': 384, 'distance': 19.112178802490234},
     {'index': 255, 'distance': 19.427560806274414}]
     
    Ele sempre retorna o mesmo item, ja que em tese ele tem distancia zero para ele mesmo - exatamente igual a ele
    mesmo =). Ele tambem retorna os itens ja em ordem de mais similar para o menos similar.
    
    """
    dict_result_query = requests.get(f'https://mycelia.azure-api.net/clone/similar/id/{DB_NAME} \ 
                                    ?index={IDX_ITEM}&top_k={TOP_K}').json()
    
    list_index_distance = dict_result_query['similarity'][0]['results']
    return list_index_distance


# -------------------------------------------------------------------------------
# -------------------------------------------------------------------------------
# -------------------------------------------------------------------------------

if __name__ == '__main__':
    TOP_K = 5 # quantidade de itens que vc quer retornar
    IDX_ITEM = 1439 # o index do item que o cliente esta olhando
    DB_NAME = 'nome_qualquer_que_vc_escolheu'
    HEADER = {'Auth': 'sua_primary_key_mycelia'}

    list_index_distance = list_k_similar_distance(db_name=DB_NAME, idx_item=IDX_ITEM, top_k=TOP_K)